In [4]:
import pandas as pd
import warnings
import sklearn as skl
warnings.filterwarnings("ignore")

In [52]:
from sklearn.metrics import classification_report, accuracy_score, mean_absolute_percentage_error

This function print the same evaluation metrics for all the methods

In [74]:
def print_eval_metric(dataset, y_true_col, y_pred_col):
    print(f'accuracy: {accuracy_score(dataset[y_true_col], dataset[y_pred_col])}')
    # print(f'Mean absolute percentage error: {mean_absolute_percentage_error(dataset[y_true_col], dataset[y_pred_col])}')
    print(classification_report(y_true=dataset[y_true_col], y_pred=dataset[y_pred_col]))

In [54]:
# Read training dataset and test dataset (unknown samples)

test_dataset = pd.read_csv('./new_files/test_dataset.csv')
training_dataset = pd.read_csv('./new_files/training_dataset.csv')

## Method 1

Here we assess the the accuracy of the first method where we determine the presence/absence of the mutation based on the signal thresholds.

In [10]:
# This file contains the Cq calculated when we keep a safe margin of atleast 100000 units of DIFFERENCE in the amplitudes between the present and absent samples.
thresholds_fixed = pd.read_csv('./new_files/thresholds_fixed.csv') 
thresholds = pd.read_csv('./new_files/thresholds_100000.csv') # This file contains the Cq calculated when we keep 100000 units of amplitude as the threshold to be crossed only.
# thresholds

,Unnamed: 0,Condition,targetID,signal_threshold,time_threshold(Cq)
0,0,ConditionA,m0,118886,71
1,1,ConditionA,m1,135502,64
2,2,ConditionA,m2,118045,30
3,3,ConditionA,m3,126319,34
4,4,ConditionA,m4,125370,53
...,...,...,...,...,...
91,91,ConditionC,m27,105867,81
92,92,ConditionC,m28,142273,70
93,93,ConditionC,m29,132455,47
94,94,ConditionC,m30,59661,73


Testing the signal thresholds

```
# thresholds.query('signal_threshold < 100000')
# thresholds.query('Condition == "ConditionA" and targetID == "m1"')['signal_threshold'].iloc[0]
# test_condA_targetm1 = test_dataset.query('condition == "ConditionA" and targetID == "m1"')
# test_condA_targetm2 = test_dataset.query('condition == "ConditionA" and targetID == "m2"')
# test_condA_targetm10 = test_dataset.query('condition == "ConditionA" and targetID == "m10"')
```

In [55]:
## Creating an unique list of condition and targets 

all_condtions = test_dataset.condition.unique()
all_targets = test_dataset.targetID.unique()

Here we create a new column called 'new_status' that determines if a mutation is absent/present based on the signal thresholds we calculated in the modelling.ipynb. We create this for both the training dataset and the test dataset. 

In [66]:
def add_new_status(test_dataset, thresholds = thresholds):
    test_dataset_new = test_dataset.copy()
    for cond in all_condtions:
        for target in all_targets:
            threshold_values = thresholds.query(f'Condition == "{cond}" and targetID == "{target}"')

            sig_threshold = threshold_values['signal_threshold'].iloc[0]
            time_threshold = threshold_values['time_threshold(Cq)'].iloc[0]

            if sig_threshold != -1 and time_threshold!= -1:
                test_dataset_new.loc[(test_dataset_new.condition==cond) & (test_dataset_new.targetID == target), 'new_status'] =  test_dataset_new.iloc[:,time_threshold].apply(lambda x : 'present' if x>= sig_threshold else 'absent')
            else:
                test_dataset_new.loc[(test_dataset_new.condition==cond) & (test_dataset_new.targetID == target), 'new_status'] = 'absent'
    return test_dataset_new

In [68]:
test_dataset_new = add_new_status(test_dataset)

In [67]:
train_dataset_new = add_new_status(training_dataset)

In [69]:
train_dataset_new_fixed = add_new_status(training_dataset, thresholds= thresholds_fixed)

In [70]:
test_dataset_new_fixed = add_new_status(test_dataset, thresholds= thresholds_fixed)

##### Comparing statuses for the known samples with fixed threshold (Method 1 Technique 1)

In [71]:
# Check the original status distribution
train_dataset_new_fixed.status.value_counts()

status
absent     2688
present     384
Name: count, dtype: int64

In [72]:
# Check the new status distribution after thresholds
train_dataset_new_fixed.new_status.value_counts()

new_status
absent     2720
present     352
Name: count, dtype: int64

In [75]:
print_eval_metric(train_dataset_new_fixed, 'status', 'new_status')

accuracy: 0.9895833333333334
              precision    recall  f1-score   support

      absent       0.99      1.00      0.99      2688
     present       1.00      0.92      0.96       384

    accuracy                           0.99      3072
   macro avg       0.99      0.96      0.98      3072
weighted avg       0.99      0.99      0.99      3072



##### Comparing statuses for the known samples with fixed DIFFERENCE threshold (Method 1 Technique 2)

In [15]:
train_dataset_new.new_status.value_counts()

new_status
absent     2728
present     344
Name: count, dtype: int64

In [24]:
print_eval_metric(train_dataset_new, 'status', 'new_status')

accuracy: 0.9869791666666666
              precision    recall  f1-score   support

      absent       0.99      1.00      0.99      2688
     present       1.00      0.90      0.95       384

    accuracy                           0.99      3072
   macro avg       0.99      0.95      0.97      3072
weighted avg       0.99      0.99      0.99      3072



Checking the original and new status for each condition

In [41]:
print_eval_metric(train_dataset_new[train_dataset_new.condition == 'ConditionA'], 'status', 'new_status')


accuracy: 1.0
              precision    recall  f1-score   support

      absent       1.00      1.00      1.00       896
     present       1.00      1.00      1.00       128

    accuracy                           1.00      1024
   macro avg       1.00      1.00      1.00      1024
weighted avg       1.00      1.00      1.00      1024



In [42]:
print_eval_metric(train_dataset_new[train_dataset_new.condition == 'ConditionB'], 'status', 'new_status')


accuracy: 0.98828125
              precision    recall  f1-score   support

      absent       0.99      1.00      0.99       896
     present       1.00      0.91      0.95       128

    accuracy                           0.99      1024
   macro avg       0.99      0.95      0.97      1024
weighted avg       0.99      0.99      0.99      1024



In [43]:
print_eval_metric(train_dataset_new[train_dataset_new.condition == 'ConditionC'], 'status', 'new_status')

accuracy: 0.97265625
              precision    recall  f1-score   support

      absent       0.97      1.00      0.98       896
     present       1.00      0.78      0.88       128

    accuracy                           0.97      1024
   macro avg       0.98      0.89      0.93      1024
weighted avg       0.97      0.97      0.97      1024



Here we see that both methods are highly accurate ( 98%) in determing the presence of the mutations based on the Cq threshold calculations only. On predicting the status of mutation presence on the known samples, we see that condition A has 100% accuracy i.e, all the known samples were correctly identified with the thresholds. We see Condition B and Condition C has lower accuracy and this is further looked into in the conclusion.

## Method 2: Machine Learning

In [1]:

import joblib
from sklearn.ensemble import VotingClassifier
from sklearn.model_selection import train_test_split

In [2]:
model_cond_all = joblib.load( filename="./new_files/model_all_data.joblib")
model_condA =joblib.load( filename="./new_files/model_condA.joblib")
model_condB =joblib.load( filename="./new_files/model_condB.joblib")
model_condC =joblib.load( filename="./new_files/model_condC.joblib")

### Comparing classifiers

Here we compare a Voting classifer made of the Decision Trees trained on each condition separately vs a combined model.

In [28]:
X = training_dataset.iloc[:,:180]
y = training_dataset.status

In [76]:
# Creating an ensemble voting classifier
ensemble_model = VotingClassifier(estimators=[
    ('tree_A', model_condA),
    ('tree_B', model_condB),
    ('tree_C', model_condC)
], voting='hard')

In [29]:
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, stratify=y, random_state=42)

In [37]:
## a Voting classifer made of the Decision Trees trained on each condition separately

ensemble_model.fit(X_train, y_train)
y_pred = ensemble_model.predict(X_test)
print(classification_report(y_true=y_test, y_pred=y_pred))
print('accuracy', accuracy_score(y_true=y_test, y_pred=y_pred))

              precision    recall  f1-score   support

      absent       0.95      0.99      0.97       538
     present       0.91      0.66      0.77        77

    accuracy                           0.95       615
   macro avg       0.93      0.83      0.87       615
weighted avg       0.95      0.95      0.95       615

accuracy 0.9495934959349593


In [77]:
# Decision TreeClassifier trained on all the data

y_pred_all = model_cond_all.predict(X_test)
print(classification_report(y_true=y_test, y_pred=y_pred_all))
print('accuracy', accuracy_score(y_true=y_test, y_pred=y_pred_all))

              precision    recall  f1-score   support

      absent       0.97      1.00      0.98       538
     present       0.98      0.78      0.87        77

    accuracy                           0.97       615
   macro avg       0.98      0.89      0.93       615
weighted avg       0.97      0.97      0.97       615

accuracy 0.9707317073170731


When comparing 2 models - a ensemble model with 3 decision trees, each specializing in a single condition vs a single decision tree that is trained on data from all the three conditions (generic tree) - we see that the generic tree performs better than the ensemble. This could be a result of hard voting mechanism where the sample is being correctly labelled by the tree condition it belongs to but fails on the other two trees. But the high accuracy (94%) suggests that the three methods have sample behaviours in common as well and that is worth further investigation.


### Testing the individual decision trees on the individual data

In [45]:
training_dataset_A = training_dataset[training_dataset.condition == 'ConditionA']
training_dataset_B = training_dataset[training_dataset.condition == 'ConditionB']
training_dataset_C = training_dataset[training_dataset.condition == 'ConditionC']

In [47]:
y_pred_A = model_condA.predict(training_dataset_A.iloc[:,:180])
y_pred_B = model_condA.predict(training_dataset_B.iloc[:,:180])
y_pred_C = model_condA.predict(training_dataset_C.iloc[:,:180])

In [49]:
# Decision Tree Results for condition A
print(classification_report(y_true=training_dataset_A.status, y_pred=y_pred_A))
print('accuracy', accuracy_score(y_true=training_dataset_A.status, y_pred=y_pred_A))

              precision    recall  f1-score   support

      absent       0.98      0.99      0.99       896
     present       0.93      0.86      0.89       128

    accuracy                           0.97      1024
   macro avg       0.96      0.93      0.94      1024
weighted avg       0.97      0.97      0.97      1024

accuracy 0.974609375


In [50]:
# Decision Tree Results for condition B

print(classification_report(y_true=training_dataset_B.status, y_pred=y_pred_B))
print('accuracy', accuracy_score(y_true=training_dataset_B.status, y_pred=y_pred_B))

              precision    recall  f1-score   support

      absent       0.97      0.86      0.91       896
     present       0.46      0.84      0.59       128

    accuracy                           0.85      1024
   macro avg       0.71      0.85      0.75      1024
weighted avg       0.91      0.85      0.87      1024

accuracy 0.8544921875


In [51]:
# Decision Tree Results for condition C

print(classification_report(y_true=training_dataset_C.status, y_pred=y_pred_C))
print('accuracy', accuracy_score(y_true=training_dataset_C.status, y_pred=y_pred_C))

              precision    recall  f1-score   support

      absent       0.96      0.95      0.95       896
     present       0.67      0.73      0.70       128

    accuracy                           0.92      1024
   macro avg       0.81      0.84      0.83      1024
weighted avg       0.92      0.92      0.92      1024

accuracy 0.9208984375


# Conclusions

## Which of the 3 assay versions is the best overall?

Considering the nature of the dataset, I would say it is key that we absolutely need to detect a mutation when it is present as it is imperative to the patient health, and a missed mutation can lead to negative patient outcomes. 

<!-- Let us consider the presence of a mutation is a postive class and the absence of mutation is the negative class. 

A false positive, i.e. the model report presence of a mutation, can be further verified using different clinical methods, and medical tests. On the other hand 

A model that is highly accurate and has a low False negative rate -->

From the threshold estimations, we can see that, both condition B and C have several samples for which the Cq could not be determined. This means that we could not find a significant time of observation that could help us conclude if a mutation is present or not for these versions. Since we assign -1 as the signal and time threshold if we can't find any, we can see the conditions and targets where we can't detect a mutation reliably -

In [39]:
thresholds_fixed.query("signal_threshold < 0")

,Unnamed: 0,Condition,targetID,signal_threshold,time_threshold(Cq)
48,48,ConditionB,m16,-1,-1
55,55,ConditionB,m23,-1,-1
64,64,ConditionC,m0,-1,-1
77,77,ConditionC,m13,-1,-1
80,80,ConditionC,m16,-1,-1
83,83,ConditionC,m19,-1,-1
85,85,ConditionC,m21,-1,-1
86,86,ConditionC,m22,-1,-1


In [40]:
thresholds.query("signal_threshold < 0")


,Unnamed: 0,Condition,targetID,signal_threshold,time_threshold(Cq)
48,48,ConditionB,m16,-1,-1
55,55,ConditionB,m23,-1,-1
58,58,ConditionB,m26,-1,-1
64,64,ConditionC,m0,-1,-1
72,72,ConditionC,m8,-1,-1
77,77,ConditionC,m13,-1,-1
80,80,ConditionC,m16,-1,-1
83,83,ConditionC,m19,-1,-1
85,85,ConditionC,m21,-1,-1
86,86,ConditionC,m22,-1,-1


Whereas for condition A, we could calculate a specific time threshold Cq for all the targets at which we can definitely detect a mutation. For condition A, we get 100% accuracy of the status detection with both threshold calculation methods, whereas for Condition B and C, we couldn't find the threshold for  several samples and these were inaccurately labelled. 

Based on Technique 1, condition A is the best assay version.

## Approach Improvements ideas


1. Feature Engineering for Machine Learning

-  Taking advantage of the temporal nature of the data by defining new features that can improve the detection of the mutation early. In this particular dataset, I would like to create new features for each sample based on a fixed time interval for observation. For example, we can calculate basic statistical representations of every 30s interval of data i.e., mean, median, skewness, kurtosis etc. that better represent the behaviour over time. 

- We can discard the 180 columns and use only these new features for training against every sample. This would reduce the computation required to train large datasets as we are dealing with significantly less features (<180). 

- A combination of these statistical features can studied across all three conditions to determine what performs best.

- The time interval can be a hyperparameter, i.e, 15/30/45/60/75/90/105 seconds etc. (either cummulative or segmented) that is created and tuned with a smaller subset of data before training the ML model. This can increase the computation costs pre-training.


2. Modelling for Machine Learning

- I have used a standard version of the Decision Tree and Ensemble model to determine the presence of mutation as reasoned in the modelling.ipynb notebook. However, this can further experimented upon with different models such as SVC which can define clear boundaries in a different space between present and absent samples.

- Clustering methods can be used with timeseries data to group and understand underlying patterns between the samples. However, due to the small size of the samples, there is a higer chance of detecting random patterns and thus this has been skipped.

3. Dimensionality Reduction

- In this modelling process, I have used two extreme methods to feature selection. One includes all the 180 seconds of observation for each sample and the other involves using only 2 statistical features. Instead of these extremes, we can explore alternative combinations of features - new and original - and reduce the dimensions of the training dataset from 180+ to a combination of orthogonal features (for example PCA) and get better results.

4. Balancing the dataset during training

- Another technique to be applied in the data preparation stage is oversampling of the samples with the mutation present or undersampling the absent samples so that we train the model on a balanced dataset that has equal split of both classes and can understand the patterns better.
